<a href="https://colab.research.google.com/github/mostafa-ja/Anomaly-detection/blob/main/BGL2_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#upload log files
!wget 'https://raw.githubusercontent.com/mostafa-ja/Anomaly-detection/main/datasets/BGL/X_train_index'
!wget 'https://raw.githubusercontent.com/mostafa-ja/Anomaly-detection/main/datasets/BGL/Xabnorm_test_index'
!wget 'https://raw.githubusercontent.com/mostafa-ja/Anomaly-detection/main/datasets/BGL/Xnorm_test_index'
!wget 'https://raw.githubusercontent.com/mostafa-ja/Anomaly-detection/main/datasets/BGL/log2index'
!wget 'https://raw.githubusercontent.com/mostafa-ja/Anomaly-detection/main/datasets/BGL/reduced_index2embed'

--2023-09-05 10:38:08--  https://raw.githubusercontent.com/mostafa-ja/Anomaly-detection/main/datasets/BGL/X_train_index
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9341136 (8.9M) [text/plain]
Saving to: ‘X_train_index’

X_train_index       100%[===================>]   8.91M  32.1MB/s    in 0.3s    

2023-09-05 10:39:12 (32.1 MB/s) - ‘X_train_index’ saved [9341136/9341136]

--2023-09-05 10:39:12--  https://raw.githubusercontent.com/mostafa-ja/Anomaly-detection/main/datasets/BGL/Xabnorm_test_index
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response..

In [2]:
import pandas as pd
import json
from sklearn.metrics.pairwise import cosine_similarity
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import os
import numpy as np
from tqdm import tqdm

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [3]:
#Json file name
with open('/content/reduced_index2embed') as f:
    embeddings = [json.loads(line) for line in f.readlines()]

len(embeddings[0])

60

In [60]:
# Define the LSTM Autoencoder model with dropout
class LSTMAutoencoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, sequence_length, dropout_prob): #input_dim= embeddings_dim
        super(LSTMAutoencoder, self).__init__()
        self.sequence_length = sequence_length
        self.encoder = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True, bidirectional=True, dropout=dropout_prob)
        self.decoder = nn.LSTM(hidden_dim , input_dim, num_layers, batch_first=True, dropout=dropout_prob)
        self.fc = nn.Linear(hidden_dim * 2, hidden_dim)

    def forward(self, x):
        encoded, _ = self.encoder(x)
        #print(encoded.shape)
        encoded = encoded[:,-1:,:] # output of last cell
        encoded = self.fc(encoded)
        #print(encoded.shape)
        input_decode = torch.tile(encoded, (1, self.sequence_length, 1))
        decoded, _ = self.decoder(input_decode)
        return decoded

In [61]:
sequence_length = 5
input_size = len(embeddings[0]) #embedding vector dimension
hidden_dim = 32
num_layers = 2
dropout_prob = 0.2

model = LSTMAutoencoder(input_size, hidden_dim, 2, sequence_length, dropout_prob).to(device)
model

LSTMAutoencoder(
  (encoder): LSTM(60, 32, num_layers=2, batch_first=True, dropout=0.2, bidirectional=True)
  (decoder): LSTM(32, 60, num_layers=2, batch_first=True, dropout=0.2)
  (fc): Linear(in_features=64, out_features=32, bias=True)
)

In [62]:
data = torch.randn(1,sequence_length, input_size) #batch_size=1
model(data.to(device)).shape

torch.Size([1, 5, 60])

In [63]:
# Calculate the number of parameters
num_params = sum(p.numel() for p in model.parameters())
print(f"Number of Parameters: {num_params}")

Number of Parameters: 103072


In [17]:
window_size = sequence_length
num_sessions = 0
sequences = {}

line = [i for i in range(20)]
print(line)

for i in range(len(line) - window_size):
    seq = tuple(line[i:i + window_size])
    if seq not in sequences:
      sequences[seq] = [ embeddings[i] for i in seq]


print(sequences.keys())

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
dict_keys([(0, 1, 2, 3, 4), (1, 2, 3, 4, 5), (2, 3, 4, 5, 6), (3, 4, 5, 6, 7), (4, 5, 6, 7, 8), (5, 6, 7, 8, 9), (6, 7, 8, 9, 10), (7, 8, 9, 10, 11), (8, 9, 10, 11, 12), (9, 10, 11, 12, 13), (10, 11, 12, 13, 14), (11, 12, 13, 14, 15), (12, 13, 14, 15, 16), (13, 14, 15, 16, 17), (14, 15, 16, 17, 18)])


In [37]:
name = 'X_train_index'
window_size = sequence_length
sequences = {}
#outputs = []
with open('/content/' + name, 'r') as f:
        for row in f:
            line = [ int(i) for i in row.strip().split()]

            for i in range(len(line) - window_size):
                seq = tuple(line[i:i + window_size])
                if seq not in sequences:
                  sequences[seq] = [ embeddings[i] for i in seq]

print('the length of dictionary : ', len(sequences))

dataset = TensorDataset(torch.tensor(list(sequences.values()), dtype=torch.float))

the length of dictionary :  24079


In [23]:
list(sequences.keys())

[(0, 0, 0, 0, 0),
 (0, 0, 0, 0, 1),
 (0, 0, 0, 1, 0),
 (0, 0, 1, 0, 0),
 (0, 0, 0, 0, 2),
 (0, 0, 0, 2, 2),
 (0, 0, 2, 2, 0),
 (0, 2, 2, 0, 2),
 (2, 2, 0, 2, 2),
 (0, 2, 2, 2, 0),
 (2, 2, 2, 0, 2),
 (2, 0, 2, 2, 2),
 (2, 2, 2, 0, 0),
 (2, 2, 0, 0, 2),
 (2, 0, 0, 2, 2),
 (0, 0, 2, 2, 2),
 (0, 2, 2, 2, 2),
 (2, 2, 2, 2, 0),
 (2, 2, 0, 2, 0),
 (2, 0, 2, 0, 0),
 (0, 2, 0, 0, 0),
 (0, 3, 4, 5, 6),
 (3, 4, 5, 6, 4),
 (4, 5, 6, 4, 4),
 (5, 6, 4, 4, 2),
 (6, 4, 4, 2, 2),
 (4, 4, 2, 2, 2),
 (4, 2, 2, 2, 2),
 (2, 2, 2, 2, 2),
 (2, 2, 2, 2, 1),
 (2, 2, 2, 1, 2),
 (2, 2, 1, 2, 2),
 (2, 1, 2, 2, 2),
 (2, 1, 7, 8, 4),
 (1, 7, 8, 4, 3),
 (7, 8, 4, 3, 3),
 (8, 4, 3, 3, 3),
 (4, 3, 3, 3, 5),
 (3, 3, 3, 5, 3),
 (3, 3, 5, 3, 5),
 (3, 5, 3, 5, 3),
 (5, 3, 5, 3, 5),
 (6, 5, 5, 5, 5),
 (5, 5, 5, 5, 5),
 (5, 5, 6, 2, 2),
 (5, 6, 2, 2, 2),
 (6, 2, 2, 2, 2),
 (2, 2, 2, 2, 9),
 (2, 2, 2, 9, 9),
 (2, 2, 9, 9, 9),
 (2, 9, 9, 9, 9),
 (9, 9, 9, 9, 9),
 (9, 9, 9, 9, 3),
 (9, 9, 9, 3, 3),
 (9, 9, 3, 3, 5),
 (9, 3, 3,

In [64]:
dataloader = DataLoader(dataset, batch_size=128, shuffle=True)
print(len(dataloader)) # regarding to batch size


for step, (seq) in enumerate(dataloader):
  print(seq[0].shape)
  break

189
torch.Size([128, 5, 60])


In [65]:
def adjust_learning_rate(optimizer, epoch, lr_step=(15,25,35), lr_decay_ratio=0.2):
    """Adjust the learning rate based on the epoch number."""
    if epoch == 0:
        optimizer.param_groups[0]['lr'] /= 8
    elif epoch in [1, 2, 3]:  # in step five , we finish warm up ,and start normal learning rate
        optimizer.param_groups[0]['lr'] *= 2
    if epoch in lr_step: # in these steps , we are geting close to optimal point so we need to have shorter step
        optimizer.param_groups[0]['lr'] *= lr_decay_ratio
    return optimizer

In [66]:
num_epochs = 40
learning_rate = 0.001

# Loss and optimizer
optimizer = optim.Adam(model.parameters(), lr=learning_rate, betas=(0.9, 0.999))
criterion = nn.MSELoss()

In [67]:
# Train the model
model.train()
start_time = time.time()
total_step = len(dataloader)
for epoch in tqdm(range(num_epochs), desc="Processing Rows"):  # Loop over the dataset multiple times
    optimizer = adjust_learning_rate(optimizer, epoch)
    train_loss = 0
    for step, (seq) in enumerate(dataloader):
        # Forward pass
        seq = seq[0].clone().detach().view(-1, window_size, input_size).to(device)
        output = model(seq)
        loss = criterion(output, seq.to(device))

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
    print('Epoch [{}/{}], train_loss: {:.4f}'.format(epoch + 1, num_epochs, train_loss / total_step))
elapsed_time = time.time() - start_time
print('elapsed_time: {:.3f}s'.format(elapsed_time))
print('Finished Training')


Processing Rows:   2%|▎         | 1/40 [00:05<03:29,  5.38s/it]

Epoch [1/40], train_loss: 0.0098


Processing Rows:   5%|▌         | 2/40 [00:11<03:35,  5.68s/it]

Epoch [2/40], train_loss: 0.0073


Processing Rows:   8%|▊         | 3/40 [00:16<03:20,  5.42s/it]

...

Epoch [37/40], train_loss: 0.0023


Processing Rows:  95%|█████████▌| 38/40 [03:26<00:10,  5.43s/it]

Epoch [38/40], train_loss: 0.0023


Processing Rows:  95%|█████████▌| 38/40 [03:31<00:11,  5.58s/it]


KeyboardInterrupt: ignored

In [ ]:
# Mount Google Drive to save datasets
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
torch.save(model.state_dict(), '/content/drive/MyDrive/BGL1_parameters.pth')

# **EVALUATION**


in cpu
```
my_model = model.load_state_dict(torch.load('model_parameters.pth', map_location=torch.device(device)))

# Put the model in evaluation mode if necessary
model.eval()
```

in cuda


```
model.load_state_dict(torch.load('model_parameters.pth'))

# Put the model in evaluation mode if necessary
model.eval()
```



In [ ]:
len(embeddings)

In [56]:
def generate(name):
    window_size = sequence_length
    hdfs = {} #store the unique sequences and their counts.
    length = 0
    with open('/content/' + name, 'r') as f:
        for row in f:
            line = [int(i) for i in row.strip().split()]
            hdfs[tuple(line)] = hdfs.get(tuple(line), 0) + 1   #If the tuple is not present in the dictionary, hdfs.get(tuple(ln), 0) returns 0, and the code initializes the count to 1.
            length += 1
            # hdfs.append(tuple(line))
    print('Number of sessions({}): {}'.format(name, len(hdfs)))
    return hdfs, length

In [57]:
test_normal_loader, test_normal_length = generate('Xnorm_test_index')
test_abnormal_loader, test_abnormal_length = generate('Xabnorm_test_index')

Number of sessions(Xnorm_test_index): 17191
Number of sessions(Xabnorm_test_index): 3595


In [58]:
def evaluation(threshold):
  # Test the model
  model.eval()

  TP = 0
  FP = 0

  start_time = time.time()
  with torch.no_grad():
      for line in tqdm(test_normal_loader.keys(), desc="Processing Rows"):
          for i in range(len(line) - window_size):
              session = line[i:i + window_size]
              seq = [embeddings[temp] for temp in session]
              seq = torch.tensor(seq, dtype=torch.float).view(-1, window_size, input_size).to(device)
              output = model(seq)

              loss = criterion(output, seq)

              if (loss.cpu().detach().numpy()>threshold):
                FP += test_normal_loader[line] # numbers of that set we have
                break
  with torch.no_grad():
      for line in tqdm(test_abnormal_loader.keys(), desc="Processing Rows"):
          for i in range(len(line) - window_size):
              session = line[i:i + window_size]
              seq = [embeddings[temp] for temp in session]
              seq = torch.tensor(seq, dtype=torch.float).view(-1, window_size, input_size).to(device)
              output = model(seq)

              loss = criterion(output, seq)

              if (loss.cpu().detach().numpy()>threshold):
                TP += test_abnormal_loader[line]
                break
  elapsed_time = time.time() - start_time
  print('elapsed_time: {:.3f}s'.format(elapsed_time))
  # Compute precision, recall and F1-measure
  FN = test_abnormal_length - TP
  P = 100 * TP / (TP + FP)
  R = 100 * TP / (TP + FN)
  F1 = 2 * P * R / (P + R)
  print('false positive (FP): {}, false negative (FN): {}, Precision: {:.3f}%, Recall: {:.3f}%, F1-measure: {:.3f}%'.format(FP, FN, P, R, F1))
  print('Finished Predicting')

In [68]:
threshold = [0.001,0.0015,0.002,0.0025]
for i in threshold:
  print('-------------------------------------------------------------------------')
  print('threshold = ',i)
  evaluation(i)

-------------------------------------------------------------------------
threshold =  0.001


Processing Rows: 100%|██████████| 3595/3595 [00:08<00:00, 438.80it/s]


elapsed_time: 61.513s
false positive (FP): 36686, false negative (FN): 26, Precision: 35.549%, Recall: 99.872%, F1-measure: 52.435%
Finished Predicting
-------------------------------------------------------------------------
threshold =  0.0015


Processing Rows: 100%|██████████| 3595/3595 [00:10<00:00, 345.55it/s]


elapsed_time: 86.630s
false positive (FP): 22678, false negative (FN): 36, Precision: 47.141%, Recall: 99.822%, F1-measure: 64.040%
Finished Predicting
-------------------------------------------------------------------------
threshold =  0.002


Processing Rows: 100%|██████████| 3595/3595 [00:13<00:00, 276.08it/s]


elapsed_time: 104.355s
false positive (FP): 21840, false negative (FN): 1127, Precision: 46.698%, Recall: 94.438%, F1-measure: 62.494%
Finished Predicting
-------------------------------------------------------------------------
threshold =  0.0025


Processing Rows:   6%|▌         | 1068/17191 [00:13<03:30, 76.48it/s]


KeyboardInterrupt: ignored

In [ ]:
threshold = [0.0057,0.0058,0.0059,0.0060,0.0061,0.0062,0.0063]
for i in threshold:
  print('-------------------------------------------------------------------------')
  print('threshold = ', i)
  evaluation(i)